In [10]:
pip install openpyxl


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   -------------- ------------------------- 92.2/250.9 kB 2.6 MB/s eta 0:00:01
   ----------------------------------- ---- 225.3/250.9 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------  245.8/250.9 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------  245.8/250.9 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------  245.8/250.9 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 1.0 MB/s eta 0:00:00


In [8]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

def first_page():
    elements = driver.find_elements(By.CSS_SELECTOR, "a[class='D113_link']")
    return [(e.text, e.get_attribute('href')) for e in elements]

def second_page(link):
    driver.get(link)
    driver.maximize_window()
    time.sleep(2)  # Ensure page is fully loaded
    routes = [(r.text, r.get_attribute('href')) for r in driver.find_elements(By.CSS_SELECTOR, "a[class='route']")]

    # Go through all available pages if present
    pages = driver.find_elements(By.CSS_SELECTOR, "div[class='DC_117_pageTabs ']")
    for p in pages:
        ActionChains(driver).click(p).perform()
        time.sleep(3)
        new_routes = driver.find_elements(By.CSS_SELECTOR, "a[class='route']")
        routes.extend((r.text, r.get_attribute('href')) for r in new_routes)
    return routes

def final_page_scraping(name, link):
    driver.get(link)
    driver.maximize_window()
    time.sleep(1)
    
    # Click on buttons if necessary
    buttons = driver.find_elements(By.CSS_SELECTOR, "div[class='button']")
    for button in reversed(buttons):
        button.click()

    # Scroll to ensure all elements are loaded
    for _ in range(10):  # Adjust the range for longer scrolling
        driver.execute_script('window.scrollBy(0, 1000)')
        time.sleep(0.5)
    
    # Extract bus information
    bus_data = []
    bus_name = driver.find_elements(By.CSS_SELECTOR, "div[class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.CSS_SELECTOR, "div[class='bus-type f-12 m-top-16 l-color evBus']")
    bus_dept = driver.find_elements(By.CSS_SELECTOR, "div[class='dp-time f-19 d-color f-bold']")
    bus_dur = driver.find_elements(By.CSS_SELECTOR, "div[class='dur l-color lh-24']")
    bus_reach = driver.find_elements(By.CSS_SELECTOR, "div[class='column-five p-right-10 w-10 fl']")
    bus_star = driver.find_elements(By.CSS_SELECTOR, "div[class='column-six p-right-10 w-10 fl']")
    bus_price = driver.find_elements(By.CSS_SELECTOR, "div[class='fare d-block']")
    bus_seats = driver.find_elements(By.CSS_SELECTOR, "div[class='column-eight w-15 fl']")

    for i in range(len(bus_name)):
        bus_data.append([
            name, link, bus_name[i].text, bus_type[i].text, bus_dept[i].text,
            bus_dur[i].text, bus_reach[i].text.split('\n')[0], bus_star[i].text.split('\n')[0],
            bus_price[i].text, bus_seats[i].text.split('\n')[0]
        ])
    
    return bus_data

# Initialize WebDriver
driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/rtc-directory')
driver.maximize_window()
time.sleep(5)

result = []
f_page = first_page()
for name, link in f_page:
    second_page_data = second_page(link)
    for route_name, route_link in second_page_data:
        bus_info = final_page_scraping(route_name, route_link)
        result.extend(bus_info)

# DataFrame creation and saving to Excel
columns = ['Bus Route Name', 'Bus route link', 'Bus Name', 'Bus Type', 'Departing time', 'Duration', 'Reaching Time', 'Rating', 'Price', 'Seats available']
df = pd.DataFrame(result, columns=columns).drop_duplicates()

# Clean and format the DataFrame
df['Departing time'] = pd.to_datetime(df['Departing time'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
df['Reaching Time'] = pd.to_datetime(df['Reaching Time'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
df['Price'] = df['Price'].str.replace('INR ', '', regex=False).astype(float)
df['Seats available'] = df['Seats available'].str.extract('(\d+)').astype(float)
df['Rating'] = pd.to_numeric(df['Rating'].str.replace('New', '').str.strip(), errors='coerce').fillna(0)

# Save to Excel
df.to_excel("output_redbus.xlsx", index=False)
print("Data extracted and saved to output_redbus.xlsx")

# Close the browser
driver.quit()


<>:83: SyntaxWarning: invalid escape sequence '\d'
<>:83: SyntaxWarning: invalid escape sequence '\d'
C:\Users\SATHISH KUMAR\AppData\Local\Temp\ipykernel_4040\4214369707.py:83: SyntaxWarning: invalid escape sequence '\d'
  df['Seats available'] = df['Seats available'].str.extract('(\d+)').astype(float)
C:\Users\SATHISH KUMAR\AppData\Local\Temp\ipykernel_4040\4214369707.py:80: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Departing time'] = pd.to_datetime(df['Departing time'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
C:\Users\SATHISH KUMAR\AppData\Local\Temp\ipykernel_4040\4214369707.py:81: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Reaching Time'] = pd.to_datetime(df['Reaching Time'], errors

ModuleNotFoundError: No module named 'openpyxl'

In [9]:
print(df)

                          Bus Route Name  \
0                     Udaipur to Jodhpur   
1                     Udaipur to Jodhpur   
2                     Udaipur to Jodhpur   
3                     Udaipur to Jodhpur   
4                     Udaipur to Jodhpur   
...                                  ...   
15270  North Lakhimpur to Nagaon (Assam)   
15284               Guwahati to Golaghat   
15285               Guwahati to Golaghat   
15288               Guwahati to Golaghat   
15294     Dibrugarh to Biswanath Charali   

                                          Bus route link  \
0      https://www.redbus.in/bus-tickets/udaipur-to-j...   
1      https://www.redbus.in/bus-tickets/udaipur-to-j...   
2      https://www.redbus.in/bus-tickets/udaipur-to-j...   
3      https://www.redbus.in/bus-tickets/udaipur-to-j...   
4      https://www.redbus.in/bus-tickets/udaipur-to-j...   
...                                                  ...   
15270  https://www.redbus.in/bus-tickets/north-lakh

In [12]:
df.to_excel("output_redbus.xlsx", index=False)
print("Data extracted and saved to output_redbus.xlsx")

Data extracted and saved to output_redbus.xlsx


In [2]:
pip install mysql-connector-python


   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.3 MB 3.3 MB/s eta 0:00:05
    --------------------------------------- 0.2/14.3 MB 2.9 MB/s eta 0:00:05
   - -------------------------------------- 0.4/14.3 MB 3.1 MB/s eta 0:00:05
   - -------------------------------------- 0.5/14.3 MB 3.4 MB/s eta 0:00:05
   - -------------------------------------- 0.7/14.3 MB 3.3 MB/s eta 0:00:05
   -- ------------------------------------- 0.8/14.3 MB 3.3 MB/s eta 0:00:05
   -- ------------------------------------- 1.0/14.3 MB 3.4 MB/s eta 0:00:04
   --- ------------------------------------ 1.2/14.3 MB 3.4 MB/s eta 0:00:04
   --- ------------------------------------ 1.4/14.3 MB 3.3 MB/s eta 0:00:04
   ---- ----------------------------------- 1.4/14.3 MB 3.3 MB/s eta 0:00:04
   ---- ----------------------------------- 1.6/14.3 MB 3.2 MB/s eta 0:00:04
   ---- ----------------------------------- 1.7/14.3 MB 3.2 MB/s eta 0:00:04
   ---


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import mysql.connector
import pandas as pd
from mysql.connector import Error

def execute_query(query, con, cursor, values=None):
    try:
        if values:
            cursor.executemany(query, values)
        else:
            cursor.execute(query)
    except Error as e:
        print(f"Error: {e}")

def create_database_and_table():
    try:
        con = mysql.connector.connect(host='localhost', user='root', password='04101998')
        cursor = con.cursor()

        # Create database and table
        queries = [
            'DROP DATABASE IF EXISTS redbus',
            'CREATE DATABASE redbus',
            'USE redbus',
            '''
            CREATE TABLE bus_routes (
                id INT AUTO_INCREMENT PRIMARY KEY,
                route_name TEXT,
                route_link TEXT,
                busname TEXT,
                bustype TEXT,
                departing_time DATETIME,
                duration TEXT,
                reaching_time DATETIME,
                star_rating FLOAT DEFAULT 0.0,
                price DECIMAL(10, 2),
                seats_available INT
            )
            '''
        ]
        for query in queries:
            execute_query(query, con, cursor)
        print("Database and Table Created")

    finally:
        if con.is_connected():
            cursor.close()
            con.close()

def insert_data_from_excel():
    try:
        con = mysql.connector.connect(host='localhost', user='root', password='04101998', database='redbus')
        cursor = con.cursor()

        # Read data from Excel
        df = pd.read_excel('output_redbus.xlsx')
        data = df.values.tolist()

        query = '''
        INSERT INTO bus_routes (
            route_name, route_link, busname, bustype, departing_time, duration,
            reaching_time, star_rating, price, seats_available
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''

        execute_query(query, con, cursor, data)

        con.commit()
        print("Values inserted successfully")

    finally:
        if con.is_connected():
            cursor.close()
            con.close()

if __name__ == "__main__":
    create_database_and_table()
    insert_data_from_excel()


Database and Table Created
Values inserted successfully
